<a href="https://colab.research.google.com/github/mohanrajmit/AgeCalculation/blob/master/CNN_AgeEstimation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Age Estimation for Indian Movie Face Dataset like Young, Old and Middle

In [1]:
!git clone https://github.com/mohanrajmit/AgeCalculation.git

Cloning into 'AgeCalculation'...
remote: Enumerating objects: 26102, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 26102 (delta 0), reused 0 (delta 0), pack-reused 26099
Receiving objects: 100% (26102/26102), 57.21 MiB | 17.09 MiB/s, done.
Resolving deltas: 100% (7437/7437), done.
Checking out files: 100% (26547/26547), done.


In [2]:
# keras imports

from keras.applications.xception import Xception, preprocess_input
from keras.applications import xception
from keras.preprocessing import image
from keras.models import Model
from keras.models import model_from_json
from keras.layers import Input
from keras.applications.inception_resnet_v2 import InceptionResNetV2

# other imports
from sklearn.preprocessing import LabelEncoder
import numpy as np
import glob
import cv2
import h5py
import os
import json
import datetime
import time
import pandas as pd




In [4]:
data=pd.read_csv("AgeCalculation/dataset/train.csv")



In [5]:
print(data.head())

          ID   Class
0    377.jpg  MIDDLE
1  17814.jpg   YOUNG
2  21283.jpg  MIDDLE
3  16496.jpg   YOUNG
4   4487.jpg  MIDDLE


In [6]:
image_name = data["ID"]
label_name = data["Class"]

features = []
labels = []

In [8]:
dataset_path="AgeCalculation/dataset/Train/"

In [9]:
for image_name1,label in zip(image_name,label_name):
	
	print("the image name  {} and label{}" .format(image_name1,label))
	image_path= str(dataset_path+image_name1)
	#print(image_path)
	#image =cv2.imread(image_path)
	#images.append(image)
	#labels.append(label)

	img = image.load_img(image_path, target_size=(73, 73))
	x = image.img_to_array(img)
	features.append(x)
	labels.append(label)

Streaming output truncated to the last 5000 lines.
the image name  11430.jpg and labelMIDDLE
the image name  14976.jpg and labelYOUNG
the image name  8698.jpg and labelMIDDLE
the image name  7492.jpg and labelMIDDLE
the image name  6134.jpg and labelMIDDLE
the image name  13508.jpg and labelMIDDLE
the image name  421.jpg and labelMIDDLE
the image name  8340.jpg and labelOLD
the image name  20983.jpg and labelMIDDLE
the image name  1284.jpg and labelMIDDLE
the image name  15799.jpg and labelMIDDLE
the image name  15439.jpg and labelMIDDLE
the image name  25435.jpg and labelMIDDLE
the image name  9079.jpg and labelMIDDLE
the image name  14915.jpg and labelMIDDLE
the image name  10023.jpg and labelMIDDLE
the image name  675.jpg and labelMIDDLE
the image name  21824.jpg and labelMIDDLE
the image name  21605.jpg and labelYOUNG
the image name  21594.jpg and labelYOUNG
the image name  20541.jpg and labelMIDDLE
the image name  7368.jpg and labelMIDDLE
the image name  3301.jpg and labelYOUNG
th

In [11]:
# encode the labels using LabelEncoder
le = LabelEncoder()
le_labels = le.fit_transform(labels)




In [12]:
le_labels

array([0, 2, 0, ..., 0, 0, 0])

In [13]:
from keras.utils import np_utils

In [14]:
list(le.inverse_transform([0, 1, 2]))

['MIDDLE', 'OLD', 'YOUNG']

In [16]:
num_classes=3

In [17]:
y_train = np_utils.to_categorical(le_labels, num_classes)

In [19]:
y_train[10]

array([0., 0., 1.], dtype=float32)

In [28]:
features=np.array(features)



In [29]:
features.shape

(19906, 73, 73, 3)

In [30]:
x_train = features.astype('float32')
x_train /= 255


In [23]:
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization

In [24]:
from keras.layers import Input
from keras.models import Model
from keras.models import Sequential

model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=(73,73,3),padding='same', activation="relu"))
model.add(Conv2D(32, (3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Conv2D(64, (3, 3), padding='same', activation="relu"))
model.add(Conv2D(64, (3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Flatten())
model.add(Dense(512, activation="relu"))
model.add(Dense(3, activation="softmax"))

In [ ]:
from __future__ import print_function

In [25]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 73, 73, 32)        896       
                                                                 
 conv2d_1 (Conv2D)           (None, 71, 71, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 35, 35, 32)       0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 35, 35, 64)        18496     
                                                                 
 conv2d_3 (Conv2D)           (None, 33, 33, 64)        36928     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 16, 16, 64)       0         
 2D)                                                  

In [26]:
model.compile(loss='categorical_crossentropy',optimizer="adam",metrics=['accuracy'])

In [31]:
model.fit(x_train,y_train,
          batch_size=32,
          epochs=10)

Epoch 1/10
623/623 [==============================] - 22s 16ms/step - loss: 0.8096 - accuracy: 0.6411
Epoch 2/10
623/623 [==============================] - 10s 16ms/step - loss: 0.6992 - accuracy: 0.7008
Epoch 3/10
623/623 [==============================] - 11s 17ms/step - loss: 0.6053 - accuracy: 0.7440
Epoch 4/10
623/623 [==============================] - 10s 16ms/step - loss: 0.4849 - accuracy: 0.8010
Epoch 5/10
623/623 [==============================] - 11s 17ms/step - loss: 0.3462 - accuracy: 0.8633
Epoch 6/10
623/623 [==============================] - 10s 16ms/step - loss: 0.1995 - accuracy: 0.9240
Epoch 7/10
623/623 [==============================] - 10s 16ms/step - loss: 0.1107 - accuracy: 0.9600
Epoch 8/10
623/623 [==============================] - 10s 16ms/step - loss: 0.0732 - accuracy: 0.9747
Epoch 9/10
623/623 [==============================] - 10s 16ms/step - loss: 0.0562 - accuracy: 0.9809
Epoch 10/10
623/623 [==============================] - 10s 16ms/step - loss: 0.054